<a href="https://colab.research.google.com/github/leandroleao2k/ImageML/blob/main/CLIP_exemplo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação CLIP do OpenAI

In [1]:
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

/bin/bash: conda: command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ri8saugg
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-ri8saugg
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369387 sha256=cfc3053848b851a43db7df0eaf216f4ebf332455bb11dc32f9f2567407b49aac
  Stored in directory: /tmp/pip-ephem-wheel-cache-npcxidyu/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [8]:
import torch
import clip
from PIL import Image

# funcao para mostrar predicoes
def format_preds(probs, texts):
    for p, t in zip(probs, texts):
      print(f'{p:6.5f}-{t}')

# importando o modelo da internet 300Mb
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# CLIP vai reconhecer o carro baseado no texto

In [10]:
from torchvision import transforms
composed = transforms.Compose([
  transforms.Resize(256),
  transforms.CenterCrop(256)
])

# carrega a imagem do notebook
image_pil = composed(Image.open("corvette.jpeg"))
image_pil

possible_classes = [
 "this car is a honda civic",
 "this car is a lamborgini",
 "this car is a corvette",
 "this car is a f40 ferrari",
]

image = preprocess(image_pil).unsqueeze(0).to(device)
text = clip.tokenize(possible_classes).to(device)

with torch.no_grad():
  image_features = model.encode_image(image)
  text_features = model.encode_text(text)

  logits_per_image, logits_per_text = model(image, text)
  probs = logits_per_image.softmax(dim=-1).cpu().numpy()

format_preds(probs[0], possible_classes)

0.00533-this car is a honda civic
0.38713-this car is a lamborgini
0.60312-this car is a corvette
0.00443-this car is a f40 ferrari


# CLIP vai avaliar as classes do texto

In [11]:
possible_classes = [
 "this is a boat",
 "this is a car",
 "this is a airplane"
]

image = preprocess(image_pil).unsqueeze(0).to(device)
text = clip.tokenize(possible_classes).to(device)

with torch.no_grad():
  image_features = model.encode_image(image)
  text_features = model.encode_text(text)

  logits_per_image, logits_per_text = model(image, text)
  probs = logits_per_image.softmax(dim=-1).cpu().numpy()

format_preds(probs[0], possible_classes)

0.00199-this is a boat
0.99477-this is a car
0.00324-this is a airplane
